In [1]:
import os
import sys
import pandas as pd
import re

import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [3]:
def data_path(fname, base_path='/home/hari/ml/dedupe-examples'):
    return base_path+'/'+fname

```bash
mysql -B -uroot -proot gem -e "select distinct p.p_brand, p.p_model, pc.pc_cat_name, pc_parent.pc_cat_name as parent_pc_cat_name, count(*) as count_occurence from products p inner join product_categories pc on pc.pc_id=p.p_cat_id inner join product_categories pc_parent on pc.pc_parent_id=pc_parent.pc_id where p.p_status=1 group by pc.pc_cat_name, p.p_brand order by pc.pc_cat_name, count_occurence desc;" > brands.tsv
```

In [4]:
brands = pd.read_csv(data_path('csv_example/brands.tsv'), sep='\t').fillna('')
brands.head(10)

,p_brand,pc_cat_name,parent_pc_cat_name,count_occurence
0,geeken,chairs-office,furniture,181
1,stellar,chairs-office,furniture,136
2,godrej,chairs-office,furniture,92
3,godrej interio,chairs-office,furniture,88
4,nilkamal,chairs-office,furniture,83
5,hans,chairs-office,furniture,76
6,vj interior,chairs-office,furniture,61
7,supreme,chairs-office,furniture,60
8,veeton,chairs-office,furniture,56
9,efc,chairs-office,furniture,52


In [5]:
grouped_brand = brands.groupby(['p_brand', 'pc_cat_name'])

In [6]:
grouped_brand_max = grouped_brand.agg({'count_occurence':'max'}).reset_index()

In [7]:
brand_max = grouped_brand_max.groupby(['p_brand']).first() \
.reset_index()\
.sort_values(by=['count_occurence'], ascending=0)

In [8]:
brand_max.to_csv(data_path('csv_example/gem_500.tsv'),sep='\t', index=False)

In [9]:
brand_max.describe()

,count_occurence
count,44144.000000
mean,2.495968
std,8.822637
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,828.000000


In [10]:
brand_max.head()

,p_brand,pc_cat_name,count_occurence
42598,voltas,Air Conditioner,828
24972,lenovo,All in one,474
20342,hp original cartriadge,printer ink cartridges,411
20343,hp original cartridge,printer ink cartridges,368
11307,dell,All in one,363


In [11]:
brand_dedup = pd.read_csv(data_path('csv_example/gem_output.csv'))
brand_dedup.head(5)

,Cluster ID,confidence_score,p_brand,pc_cat_name,count_occurence,canonical_count_occurence,canonical_pc_cat_name,canonical_p_brand
0,0,0.576558,voltas,Air Conditioner,828,10.0,air conditioner,voltas
1,1,0.564770,lenovo,All in one,474,15.0,desktops,lenovo
2,2,0.025881,hp original cartriadge,printer ink cartridges,411,104.0,printer ink cartridges,hp
3,2,0.028977,hp original cartridge,printer ink cartridges,368,104.0,printer ink cartridges,hp
4,92,0.535665,dell,All in one,363,11.0,desktops,dell


In [15]:
len(brand_dedup['Cluster ID'].unique())

15449

In [16]:
len(brand_dedup)

44143

In [17]:
b = brands
b['all_categories'] = b['pc_cat_name']

In [18]:
z = b.groupby(['p_brand'])['all_categories'].apply(lambda x:" | ".join(x)).reset_index()

In [19]:
z.head()

,p_brand,all_categories
0,,Chappals | hospital biohazard bags or bio medi...
1,&#x28;colin&#x29;&#x28;500ml&#x29;,glass or window cleaners
2,&#x28;dettol&#x29;,liquid handwash
3,&#x28;gainda&#x29;&#x28;5ltr&#x29;,phenyl
4,&#x28;hp-kay pee enterprises &#x28;hp rsr&#x29;,printer ink cartridges


In [24]:
brand_other_cat = z

In [25]:
result = pd.merge(brand_dedup, brand_other_cat, on=['p_brand'])

In [26]:
len(result)

44143

In [27]:
result.head(3)

,Cluster ID,confidence_score,p_brand,pc_cat_name,count_occurence,canonical_count_occurence,canonical_pc_cat_name,canonical_p_brand,all_categories
0,0,0.576558,voltas,Air Conditioner,828,10.0,air conditioner,voltas,Air Conditioner | automatic line voltage corre...
1,1,0.564770,lenovo,All in one,474,15.0,desktops,lenovo,All in one | bag kit universal | bar code read...
2,2,0.025881,hp original cartriadge,printer ink cartridges,411,104.0,printer ink cartridges,hp,printer ink cartridges


In [29]:
clusters = result.sort_values(['Cluster ID']).reset_index(drop=True)

In [32]:
# clusters.to_csv(data_path('csv_example/gem_output_merged.tsv'),sep='\t', index=False)

# Clustered output

In [33]:
clusters.head(4)

,Cluster ID,confidence_score,p_brand,pc_cat_name,count_occurence,canonical_count_occurence,canonical_pc_cat_name,canonical_p_brand,all_categories
0,0,0.576558,voltas,Air Conditioner,828,10.0,air conditioner,voltas,Air Conditioner | automatic line voltage corre...
1,0,0.119077,voltas 1 ton window ac,Air Conditioner,1,10.0,air conditioner,voltas,Air Conditioner
2,0,0.140414,voltas 15 ml,Drinking Water Cooler,1,10.0,air conditioner,voltas,Drinking Water Cooler
3,0,0.143160,voltas - 5 star,Air Conditioner,1,10.0,air conditioner,voltas,Air Conditioner


In [34]:
len(clusters)

44143

In [35]:
clusters['cluster_count'] = 1

In [36]:
grouped_clusters = clusters.groupby('Cluster ID')

In [37]:
len(grouped_clusters)

15449

In [38]:
cluster_count_df = grouped_clusters.agg({'cluster_count': sum}).reset_index()

In [60]:
clusters.drop('cluster_count', axis=1, inplace=True)
cluster_with_count = pd.merge(clusters, cluster_count_df, on=['Cluster ID'])

In [85]:
cluster_with_count['cluster_count'] = cluster_with_count['cluster_count'].apply(int)

In [102]:
def cleanse_brand(column):
    if column:
        column = column.lower()
        column = re.sub(' by .*', ' ', column)
        column = re.sub('&#.*;', ' ', column)
        column = re.sub('[^0-9a-z ]', ' ', column)
        column = column.strip()
    return column

cluster_with_count['brand_cleansed'] = cluster_with_count['p_brand'].apply(cleanse_brand)

In [103]:
cluster_with_count.head(2)

,Cluster ID,confidence_score,p_brand,pc_cat_name,count_occurence,canonical_count_occurence,canonical_pc_cat_name,canonical_p_brand,all_categories,cluster_count,brand_cleansed
0,0,0.666343,voltas,Air Conditioner,828,1.0,air conditioner,voltas,Air Conditioner | automatic line voltage corre...,27,voltas
1,0,0.676081,"voltas-,-",Air Conditioner,5,1.0,air conditioner,voltas,Air Conditioner,27,voltas


In [104]:
single = cluster_with_count[cluster_with_count['cluster_count'] == 1]
len(single)

19716

In [99]:
family = cluster_with_count[cluster_with_count['cluster_count'] != 1]
len(family)

24427

In [100]:
single.to_csv('/root/dedupe-examples/csv_example/single.tsv',sep='\t', index=False)
single.head(2)

,Cluster ID,confidence_score,p_brand,pc_cat_name,count_occurence,canonical_count_occurence,canonical_pc_cat_name,canonical_p_brand,all_categories,cluster_count,brand_cleansed
24427,4831,NaN,addison,button bits for dth hammer for water well dril...,279,NaN,NaN,NaN,button bits for dth hammer for water well dril...,1,addison
24428,4832,NaN,lushomes,bathroom mat,191,NaN,NaN,NaN,bathroom mat | bedclothes rubberised coir matt...,1,lushomes


In [101]:
family.to_csv('/root/dedupe-examples/csv_example/family.tsv',sep='\t', index=False)
family.head(2)

,Cluster ID,confidence_score,p_brand,pc_cat_name,count_occurence,canonical_count_occurence,canonical_pc_cat_name,canonical_p_brand,all_categories,cluster_count,brand_cleansed
0,0,0.666343,voltas,Air Conditioner,828,1.0,air conditioner,voltas,Air Conditioner | automatic line voltage corre...,27,voltas
1,0,0.676081,"voltas-,-",Air Conditioner,5,1.0,air conditioner,voltas,Air Conditioner,27,voltas


# spurious

### Cleaners
* remove numbers

### Filters
* length < 3
* no vowel single word




In [135]:
re.sub(r'\b[0-9]+\b', '', 'Hello 1200')

'Hello '

In [136]:
def remove_number(s):
    if s:
        s = re.sub(r'\b[0-9]+\b', '', s)
        s = re.sub(r'\s+', ' ', s)
        s = s.strip()
    return s

def length_less_than(s, n=3):
    if s:
        return len(s) < n
    else:
        return True

def no_vowel(s):
    for v in 'aeiou':
        if v in s:
            return False
    return True

def single_word(s):
    if s:
        return not ' ' in s
    return False

In [137]:
remove_number('Hello world 32423 4324243 gerigje 234')

'Hello world gerigje'

In [138]:
single_word('gggggg') and no_vowel('gggggg')

True

In [151]:
single['brand_cleansed'] = single['brand_cleansed'].apply(remove_number)

single['spurious'] = single['brand_cleansed'].apply(lambda x: single_word(x) and no_vowel(x))

In [152]:
len(single[single['spurious']])

1722

In [153]:
len(single[~single['spurious']])

17994

In [154]:
single.keys()

Index([u'Cluster ID', u'confidence_score', u'p_brand', u'pc_cat_name',
       u'count_occurence', u'canonical_count_occurence',
       u'canonical_pc_cat_name', u'canonical_p_brand', u'all_categories',
       u'cluster_count', u'brand_cleansed', u'spurious'],
      dtype='object')

In [155]:
single[~single['spurious']][['p_brand', 'brand_cleansed', 'pc_cat_name']]

,p_brand,brand_cleansed,pc_cat_name
24427,addison,addison,button bits for dth hammer for water well dril...
24428,lushomes,lushomes,bathroom mat
24429,h p - pioneer enterprises rsr dli,h p pioneer enterprises rsr dli,printer ink cartridges
24430,hp - sri jai media rsr,hp sri jai media rsr,printer ink cartridges
24431,radex,radex,Computer Paper
24432,canon-ascent-authorised reseller,canon ascent authorised reseller,printer ink cartridges
24433,whale,whale,Computer Paper
24434,pixaarnext,pixaarnext,Desktops
24435,heera,heera,Computer Paper
24436,sophos,sophos,networking devices
